# Visualizing data with 3D model

In [ ]:
# Import necessary pyologger utilities
from pyologger.utils.folder_manager import *
from pyologger.utils.event_manager import *
from pyologger.plot_data.plotter import *
from pyologger.io_operations.base_exporter import *
from pyologger.utils.data_manager import *
from pyologger.process_data.peak_detect import *

dataset_id = "oror-adult-orca_hr-sr-vid_sw_JKB-PP"
deployment_id = "2024-01-16_oror-002"

# Load important file paths and configurations
config, data_dir, color_mapping_path, montage_path = load_configuration()
# Streamlit load data
animal_id, dataset_id, deployment_id, dataset_folder, deployment_folder, data_pkl, config_manager = select_and_load_deployment(data_dir, dataset_id=dataset_id, deployment_id=deployment_id)
pkl_path = os.path.join(deployment_folder, 'outputs', 'data.pkl')

In [ ]:
media_path = os.path.join(config['paths']['local_private_media'], dataset_id, deployment_id, 'media', '02_processed-video')
media_path

In [ ]:
import vidformer as vf
import vidformer.cv2 as cv2

server = vf.Server("http://localhost:8080", api_key = "VF_IGNI_API_KEY")

cap = cv2.VideoCapture(media_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("my_output.mp4", cv2.VideoWriter_fourcc(*"mp4v"),
                        fps, (width, height))
while True:
    ret, frame = cap.read()
    if not ret:
      break

    cv2.putText(frame, "Hello, World!", (100, 100), cv2.FONT_HERSHEY_SIMPLEX,
                1, (255, 0, 0), 1)
    out.write(frame)

cap.release()
out.release()

In [ ]:
# Initialize the info class
metadata = Metadata()
metadata.fetch_databases(verbose=False)

# Save databases
deployment_db = metadata.get_metadata("deployment_DB")
logger_db = metadata.get_metadata("logger_DB")
recording_db = metadata.get_metadata("recording_DB")
animal_db = metadata.get_metadata("animal_DB")

# Assuming you have the metadata and deployment_db loaded:
datareader = DataReader()
deployment_folder, deployment_id = datareader.check_deployment_folder(deployment_db, data_dir)
config_manager = ConfigManager(deployment_folder=deployment_folder, deployment_id=deployment_id)

In [ ]:
# Load the data_reader object from the pickle file
pkl_path = os.path.join(deployment_folder, 'outputs', 'data.pkl')

with open(pkl_path, 'rb') as file:
    data_pkl = pickle.load(file)

for logger_id, info in data_pkl.logger_info.items():
    sampling_frequency = info.get('datetime_metadata', {}).get('fs', None)
    if sampling_frequency is not None:
        # Format the sampling frequency to 5 significant digits
        print(f"Sampling frequency for {logger_id}: {sampling_frequency} Hz")
    else:
        print(f"No sampling frequency available for {logger_id}")

In [ ]:
# Retrieve values from config
variables = ["calm_horizontal_start_time", "calm_horizontal_end_time", 
             "zoom_window_start_time", "zoom_window_end_time", 
             "stroking_start_time", "stroking_end_time",
             "overlap_start_time", "overlap_end_time"]
settings = config_manager.get_from_config(variables, section="settings")

# Assign retrieved values to variables
CALM_HORIZONTAL_START_TIME = settings.get("calm_horizontal_start_time")
CALM_HORIZONTAL_END_TIME = settings.get("calm_horizontal_end_time")
ZOOM_START_TIME = settings.get("zoom_window_start_time")
ZOOM_END_TIME = settings.get("zoom_window_end_time")
OVERLAP_START_TIME = settings.get("overlap_start_time")
OVERLAP_END_TIME = settings.get("overlap_end_time")

In [ ]:
TARGET_SAMPLING_RATE = 10

notes_to_plot = {
    'heartbeat_manual_ok': {'signal': 'hr_normalized', 'symbol': 'triangle-down', 'color': 'blue'},
    'heartbeat_auto_detect_accepted': {'signal': 'hr_normalized', 'symbol': 'triangle-up', 'color': 'green'},
    'strokebeat_auto_detect_accepted': {'signal': 'sr_smoothed', 'symbol': 'triangle-up', 'color': 'green'},
}

fig = plot_tag_data_interactive(
    data_pkl=data_pkl,
    sensors=['ecg', 'hr_normalized'],
    derived_data_signals=['depth', 'prh', 'stroke_rate', 'heart_rate','sr_smoothed', 'odba'],
    channels={}, #'corrected_gyr': ['broad_bandpassed_signal']
    time_range=(OVERLAP_START_TIME, OVERLAP_END_TIME),
    note_annotations=notes_to_plot,
    color_mapping_path=color_mapping_path,
    target_sampling_rate=TARGET_SAMPLING_RATE,
    zoom_start_time=ZOOM_START_TIME,
    zoom_end_time=ZOOM_END_TIME,
    zoom_range_selector_channel='depth',
    plot_event_values=[],
)

fig.show()

In [ ]:
# Retrieve timezone from deployment info
timezone = data_pkl.deployment_info['Time Zone']

VIDEO_FILENAME = "2024-01-16_oror-002a_CC-96_095653-101244.mp4"

# Define placeholder timestamps for calm period in the retrieved timezone
VIDEO_START_TIME = pd.Timestamp("2024-01-16 09:56:53").tz_localize(timezone)
VIDEO_END_TIME = pd.Timestamp("2024-01-16 10:12:44").tz_localize(timezone)

# Use ConfigManager to add both stroking start and end times to the config in the desired section
config_manager.add_to_config(
    entries={
        "video_filename": str(VIDEO_FILENAME),
        "video_start_time": str(VIDEO_START_TIME),
        "video_end_time": str(VIDEO_END_TIME)
    },
    section="settings"
)

In [ ]:
VIDEO_START_TIME

In [ ]:
VIDEO_END_TIME

In [ ]:
 df_all = data_pkl.derived_data['prh']

# Convert to UTC
#dff["datetime"] = dff["datetime"].dt.tz_localize("UTC")
# convert the datetime in dff to timezone
#dff["datetime"] = dff["datetime"] + pd.Timedelta(hours=-8)
print(f'')

dff = df_all[(df_all['datetime'] >= VIDEO_START_TIME) & (df_all['datetime'] <= VIDEO_END_TIME)]
dff.reset_index(drop=True, inplace=True)
# Convert datetime to timestamp (seconds since epoch) for slider control
dff["timestamp"] = dff["datetime"].apply(lambda x: x.timestamp())

In [ ]:
dff

In [ ]:
# Process video data
import cv2
import pytesseract
from datetime import datetime

# Path to Tesseract executable (modify as per your installation)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_timestamps(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    timestamps = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Crop bottom right corner (adjust these values to match the location in your video)
        h, w, _ = frame.shape
        cropped = frame[int(h*0.85):h, int(w*0.7):w]

        # Convert to grayscale
        gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)

        # Perform OCR
        text = pytesseract.image_to_string(gray, config='--psm 6')
        text = text.strip()

        try:
            # Parse timestamp (adjust format as needed)
            timestamp = datetime.strptime(text, '%d%b%y %H:%M:%S.%f')
            timestamps.append((frame_count, timestamp))
        except ValueError:
            pass  # Skip if OCR output is not a valid timestamp

    cap.release()
    return timestamps

# Usage
video_path = 'your_video.mp4'
timestamps = extract_timestamps(video_path)
print(timestamps)


In [ ]:
import plotly.offline as pyo

# Specify the file path to save the HTML file
html_file_path = os.path.join(deployment_folder,'hr_data.html')

# Save the fig plot as an HTML file
pyo.plot(fig, filename=html_file_path, auto_open=False)

In [ ]:
import os
import trimesh
import numpy as np
from pythreejs import *
from IPython.display import display

# Define the path to the OBJ file
model_path = os.path.join(data_dir, '/assets/', '6_killerWhale_v017_LP.obj')

# Load the OBJ file using Trimesh
mesh = trimesh.load(model_path)

# Extract vertices and faces from the mesh
vertices = mesh.vertices
faces = mesh.faces.astype(np.uint32)  # Ensure the indices are unsigned integers

# Create BufferGeometry for PyThreeJS
geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(vertices, normalized=False),
        'index': BufferAttribute(faces.flatten(), normalized=False)
    }
)

# Create a material
material = MeshLambertMaterial(color='red') # , roughness=0.9, metalness=0.2
key_light = DirectionalLight(color='white', position=[3, 5, 1], intensity=0.75)

# Create the Mesh
mesh = Mesh(geometry=geometry, material=material, position=[0, 0, 200], rotation= [0, math.pi, 0, 'XYZ'])

# Move (translate) the mesh
mesh.position = [0, 0, 200]  # Move the mesh up by 1 unit on the Y-axis

# Rotate the mesh
#mesh.rotation = [0.5, 0.5, 0, 0]  # Rotate the mesh around the X and Y axes (in radians)

# Scale the mesh
mesh.scale = [1.5, 1.5, 1.5]  # Scale the mesh uniformly by 1.5 on all axes

# Create a scene, camera, and renderer
scene = Scene(children=[mesh, AmbientLight(color='#ffffff', intensity=0.5)])
camera = PerspectiveCamera(position=[0, 0, 5], up=[0, 1, 0], children=[key_light])
renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)], width=400, height=400)

# Display the model in the notebook
display(renderer)

ball = Mesh(geometry=SphereGeometry(), 
            material=MeshLambertMaterial(color='red'))


c = PerspectiveCamera(position=[0, 5, 5], up=[0, 1, 0], children=[key_light])

scene = Scene(children=[ball, c, AmbientLight(color='#777777')], background=None)

renderer = Renderer(camera=c, 
                    scene=scene,
                    alpha=True,
                    clearOpacity=0,
                    controls=[OrbitControls(controlling=c)])
display(renderer)